In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# matplotlib 폰트설정
# plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rc('font', family='Malgun Gothic') # For Windows
print(plt.rcParams['font.family'])

%matplotlib inline
# 브라우저에서 바로 이미지를 그린다.v

['Malgun Gothic']


In [2]:
# 지수표현 제거
pd.options.display.float_format = '{:.5f}'.format

# 1. cluster

In [3]:
# 기존에 log데이터 전처리한거 로드
log_cluster = pd.read_csv('data/new_log_event_cluster.csv')
log_cluster.head()

,user_id,date,time,cluster
0,1,2022-05-03,7,1
1,1,2022-06-16,1,1
2,7,2022-05-22,0,1
3,9,2022-05-21,354,1
4,11,2022-03-24,692,1


## 1.1. cluster 종류
- 0. 기타
- 1. 부가서비스(GetCreditInfo)
- 2. start - end
- 3. 회원가입
- 4. login
- 5. openapp

## 1.2 log데이터에서 user 통일
- time : 누적합
- cluster : 한 user가 여러 개의 cluster 가지고 있으면, 가장 많은 것 기준으로
- count : user별 row 기준으로 count

In [4]:
type(log_cluster[log_cluster['user_id'] == 1]['time'].sum())

numpy.int64

In [5]:
len(log_cluster[log_cluster['user_id'] == 1])

2

In [6]:
# log데이터에서 user_id 몇개인지 추출
user_id_list = log_cluster['user_id'].unique()
print(len(user_id_list))

582133


In [7]:
from tqdm import tqdm

In [8]:
date_list = []
time_list = []
cluster_list = []
count_list = []

for i in tqdm(user_id_list):
    # 특정 user_id를 가진 user 추출
    user_rows = log_cluster[log_cluster['user_id'] == i]
    
    # 시간은 user이 총 사용했던 시간 누적합
    time = user_rows['time'].sum()
    
    # cluster은 가장 많은 cluster 기준으로 분류
    cluster = user_rows['cluster'].value_counts().index[0]
    
    # 가장 마지막 날짜 기준으로
    date = user_rows['date'].iloc[-1]
    
    
    # 한 유저당 어플 사용 횟수(15분 기준)
    count = len(user_rows)
    
    time_list.append(time)
    cluster_list.append(cluster)
    date_list.append(date)
    count_list.append(count)
    

    
        
    

100%|██████████| 582133/582133 [1:08:05<00:00, 142.49it/s]


In [9]:
# 위에서 추출했던 데이터 가지고 새로운 데이터프레임 만들기
log_cluster_final = pd.DataFrame({'user_id': user_id_list,
              'date' : date_list,
              'time' : time_list,
              'cluster' : cluster_list,
              'count': count_list
             })
log_cluster_final.head()

,user_id,date,time,cluster,count
0,1,2022-06-16,8,1,2
1,7,2022-05-22,0,1,1
2,9,2022-05-21,354,1,1
3,11,2022-06-13,1687,1,5
4,12,2022-06-30,2744,2,15


In [10]:
# log_cluster_final.csv이라는 이름으로 최종 저장
log_cluster_final.to_csv('data/log_cluster_final.csv', index=False)